In [9]:
import pandas as pd
import plotly
import plotly.express as px
import statsmodels.api as sm
import numpy as np


df = pd.read_csv("/kaggle/input/hpindex/HPI_master.csv")
check = df[["place_name","yr","period","index_nsa"]]
check = check[check.place_name=="East North Central Division"]

In [3]:
check

,place_name,yr,period,index_nsa
0,East North Central Division,1991,1,100.00
1,East North Central Division,1991,2,101.03
2,East North Central Division,1991,3,101.40
3,East North Central Division,1991,4,101.79
4,East North Central Division,1991,5,102.44
...,...,...,...,...
92964,East North Central Division,2015,2,187.55
92965,East North Central Division,2015,3,189.40
92966,East North Central Division,2015,4,187.96
92967,East North Central Division,2016,1,188.42


In [4]:
import pandas as pd

def create_shifted_df(df, target_col, date_range, shift_values):
    # Extract the start and end dates from the date range
    start_date = pd.to_datetime(f'01-{date_range[0]}', format='%d-%m-%Y')
    end_date = pd.to_datetime(f'01-{date_range[1]}', format='%d-%m-%Y')

    # Create a date range with all months between start_date and end_date
    all_months = pd.date_range(start=start_date, end=end_date, freq='MS').strftime('%Y-%m').tolist()

    # Create a new DataFrame with the month column
    new_df = pd.DataFrame(all_months, columns=['month'])

    # Merge the original dataframe with the new dataframe on the month column
    new_df = new_df.merge(df[['month', target_col]], on='month', how='left')

    # Create shifted columns based on the list of integers
    for shift in shift_values:
        new_df[f'{target_col}_shift_{shift}'] = new_df[target_col].shift(shift)

    return new_df

In [5]:
# Example usage:
# Assuming 'df' is your initial DataFrame, 'target_col' is the column to be shifted,
# 'date_range' is a tuple of the start and end dates, and 'shift_values' is a list of shifts.
df = pd.DataFrame({
    'month': ['2023-01', '2023-02', '2023-03', '2023-04', '2023-05'],
    'target': [10, 15, 20, 25, 30]
})
target_col = 'target'
date_range = ('01-2023', '12-2023')
shift_values = [-1, 1]

result_df = create_shifted_df(df, target_col, date_range, shift_values)

In [6]:
result_df

,month,target,target_shift_-1,target_shift_1
0,2023-01,10.0,15.0,NaN
1,2023-02,15.0,20.0,10.0
2,2023-03,20.0,25.0,15.0
3,2023-04,25.0,30.0,20.0
4,2023-05,30.0,NaN,25.0
5,2023-06,NaN,NaN,30.0
6,2023-07,NaN,NaN,NaN
7,2023-08,NaN,NaN,NaN
8,2023-09,NaN,NaN,NaN
9,2023-10,NaN,NaN,NaN


In [10]:
def plot_and_regress(df, month_col, variable_col, target_col):
    # Ensure the month column is in datetime format
    df[month_col] = pd.to_datetime(df[month_col])
    
    # Sort the DataFrame by month
    df = df.sort_values(by=month_col)
    
    # Plot the variable and target month-wise using plotly
    fig_1 = px.line(df, x=month_col, y=[variable_col, target_col], 
                  labels={month_col: 'Month', 'value': 'Value'},
                  title=f'{variable_col} and {target_col} over Time')

    # Perform univariate regression using statsmodels
    X = df[[variable_col]]
    X = sm.add_constant(X)  # Add a constant term for the intercept
    Y = df[target_col]
    
    model = sm.OLS(Y, X).fit()
    
    # Show regression results
    print(model.summary())
    
    # Predict the target values
    df['prediction'] = model.predict(X)
    
    # Plot the actual and predicted target values
    fig_2 = px.line(df, x=month_col, y=[target_col, 'prediction'], 
                  labels={month_col: 'Month', 'value': 'Value'},
                  title=f'Actual and Predicted {target_col} over Time')
    return [fig_1, fig_2, model]

In [11]:
# Create a larger dataset
date_rng = pd.date_range(start='1/1/2010', end='12/31/2023', freq='ME')
df = pd.DataFrame(date_rng, columns=['month'])
df['variable'] = np.random.randint(1, 100, size=(len(date_rng)))
df['target'] = df['variable'] + np.random.normal(0, 10, size=(len(date_rng)))

month_col = 'month'
variable_col = 'variable'
target_col = 'target'

Test_2 = plot_and_regress(df, month_col, variable_col, target_col)

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.898
Model:                            OLS   Adj. R-squared:                  0.897
Method:                 Least Squares   F-statistic:                     1459.
Date:                Mon, 18 Nov 2024   Prob (F-statistic):           3.79e-84
Time:                        07:39:51   Log-Likelihood:                -610.03
No. Observations:                 168   AIC:                             1224.
Df Residuals:                     166   BIC:                             1230.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5566      1.521      0.366      0.7

In [16]:
import pandas as pd
import numpy as np
import joblib

def fill_target_column_with_pretrained_model(df, month_col, target_col, model_path):
    # Convert month column to datetime format
    df[month_col] = pd.to_datetime(df[month_col], format='%m-%Y')
    
    # Sort the DataFrame by month
    df = df.sort_values(by=month_col)
    
    # Load the pre-trained model
    model = joblib.load(model_path)
    
    # Get the feature names from the model
    feature_names = model.feature_names_in_
    
    # Iterate over the DataFrame to fill missing target values
    for i in range(1, len(df)):
        # Ensure the first row's target value is filled
        if pd.isna(df[target_col].iloc[i-1]):
            raise ValueError("The first row's target value must be provided.")
        
        # Features are all columns except the target and month columns
        features = df.drop(columns=[month_col, target_col]).iloc[:i]
        X_next = features.values
        
        # Ensure the feature names match those used during training
        features_next = pd.DataFrame([X_next[-1]], columns=feature_names)
        
        # Predict the next target value using the pre-trained model
        df.loc[i, target_col] = model.predict(features_next)[0]
    
    return df

# Create a larger dataset
date_rng = pd.date_range(start='1/1/2010', end='12/31/2023', freq='ME')
df = pd.DataFrame(date_rng, columns=['month'])
df['var1'] = np.random.randint(1, 100, size=(len(date_rng)))
df['var2'] = np.random.randint(1, 100, size=(len(date_rng)))
df['target'] = np.nan
df.loc[0, 'target'] = 50  # Set the first target value

month_col = 'month'
target_col = 'target'
model_path = 'pretrained_model.pkl'

# Create and save a demo pre-trained model
train_data = {
    'var1': np.random.randint(1, 100, size=100),
    'var2': np.random.randint(1, 100, size=100),
    'target': np.random.randint(1, 100, size=100)
}
train_df = pd.DataFrame(train_data)
model = LinearRegression()
X_train = train_df[['var1', 'var2']]
y_train = train_df['target']
model.fit(X_train, y_train)
model.feature_names_in_ = X_train.columns  # Save feature names in the model

# Save the trained model to a file
joblib.dump(model, model_path)

# Fill the target column using the pre-trained model
filled_df = fill_target_column_with_pretrained_model(df, month_col, target_col, model_path)
print(filled_df)


         month  var1  var2     target
0   2010-01-31    92    17  50.000000
1   2010-02-28    70    64  41.259524
2   2010-03-31    43     4  49.059069
3   2010-04-30    69    85  46.408526
4   2010-05-31    66    76  51.372816
..         ...   ...   ...        ...
163 2023-08-31    29    36  44.863195
164 2023-09-30    90    37  51.589039
165 2023-10-31    95    79  43.602046
166 2023-11-30    88    97  47.301080
167 2023-12-31    90    57  50.099062

[168 rows x 4 columns]


In [19]:
import pandas as pd

def apply_model_to_row(row, model_name_col, models_dict):
    # Get the model name from the specified column in the row
    model_name = row[model_name_col]
    
    # Ensure the model name exists in the dictionary
    if model_name not in models_dict:
        raise ValueError(f"Model '{model_name}' not found in models dictionary.")
    
    # Get the specified model
    model = models_dict[model_name]
    
    # Drop the model name column to get the features for prediction
    features = row.drop(labels=[model_name_col]).values.reshape(1, -1)
    
    # Predict the target value using the specified model
    target_prediction = model.predict(features)[0]
    
    return target_prediction

# Example usage
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import numpy as np

# Create some example models
model1 = LinearRegression()
model2 = DecisionTreeRegressor()

# Dummy training data for models (Normally, you would load pre-trained models)
X_train = np.random.rand(100, 3)
y_train = np.random.rand(100)
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)

# Dictionary of models
models_dict = {
    'CC': model1,
    'C3': model2
}

# Example row with model_name
example_row = pd.Series({
    'var1': 0.5,
    'var2': 0.6,
    'var3': 0.7,
    'model_name': 'CC'
})

# Apply the function to predict the target value for the example row
target_prediction = apply_model_to_row(example_row, 'model_name', models_dict)
print(f'Predicted target value: {target_prediction}')

Predicted target value: 0.4198019965672751


In [18]:
example_row

var1                        0.5
var2                        0.6
var3                        0.7
model_name    linear_regression
dtype: object